In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist
from sklearn.metrics import davies_bouldin_score
from sklearn.neighbors import NearestNeighbors

customers = pd.read_csv("C:/Users/Lahari/Downloads/Customers.csv")
products = pd.read_csv("C:/Users/Lahari/Downloads/Products.csv")
transactions = pd.read_csv("C:/Users/Lahari/Downloads/Transactions.csv")



# Now re-run your code
customer_features = transactions.groupby('CustomerID').agg({'TotalValue': 'sum', 'Quantity': 'sum'}).reset_index()
customer_data = customers.merge(customer_features, on='CustomerID', how='left').fillna(0)


scaler = StandardScaler()
scaled_data = scaler.fit_transform(customer_data[['TotalValue', 'Quantity']])

nn = NearestNeighbors(n_neighbors=4, metric='euclidean')
nn.fit(scaled_data)

def find_lookalikes(customer_id):
    idx = customer_data[customer_data['CustomerID'] == customer_id].index[0]
    distances, indices = nn.kneighbors([scaled_data[idx]])
    lookalikes = [(customer_data.iloc[i]['CustomerID'], distances[0][j]) for j, i in enumerate(indices[0][1:])]
    return lookalikes

lookalike_results = {cust_id: find_lookalikes(cust_id) for cust_id in customers['CustomerID'][:20]}
lookalike_df = pd.DataFrame.from_dict(lookalike_results, orient='index', columns=['Lookalike1', 'Lookalike2', 'Lookalike3'])
lookalike_df.to_csv('lahari_kondi_Lookalike.csv')
